In [1]:
import os
import numpy as np
import glob

import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dropout, Input
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
train_x = np.load('./train_375.npy')
test_x = np.load('./test_125.npy')

In [2]:
#train_a = np.zeros(shape=(1,61,61,275,1))
#for f in glob.glob("./train/*.npy"):
#    with open(f, "rb") as infile:
#            train_data = np.load(infile).reshape(1,61,61,275,1)
#            train_b = np.concatenate([train_data,train_a],axis=0)
#            train_a = train_b

KeyboardInterrupt: 

In [ ]:
#train_b.shape

In [ ]:
#train_x = np.delete(train_b,375,0)

In [ ]:
#np.save('./train_375.npy',train_x)

In [ ]:
#test1 = np.zeros(shape=(1,61,61,275,1))
#for f in glob.glob("./test/*.npy"):
#    with open(f, "rb") as infile:
#            test = np.load(infile).reshape(1,61,61,275,1)
#            test2 = np.concatenate([test,test1],axis=0)
#            test1 = test2

In [3]:
test_x.shape

(125, 61, 61, 275, 1)

In [ ]:
#test_x = np.delete(test2,125,0)

In [ ]:
#np.save('./test_125.npy',test_x)

In [4]:
import pandas as pd

label_df = pd.read_csv("./germination_label.csv")
label_df.head(5)

,0
0,0
1,0
2,1
3,0
4,0


In [5]:
train_y = label_df[0:375]
train_y.head(5)

,0
0,0
1,0
2,1
3,0
4,0


In [6]:
from keras.utils import to_categorical
train_y = to_categorical(train_y)

In [7]:
test_y = label_df[375:500]
test_y.head(5)

,0
375,1
376,0
377,1
378,0
379,1


In [8]:
test_y = to_categorical(test_y)
test_y

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [9]:
input_layer = Input((61, 61, 275,1))

conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 7), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 5), activation='relu')(conv_layer1)
conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(conv_layer2)
print(conv_layer3._keras_shape)
conv3d_shape = conv_layer3._keras_shape
conv_layer3 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layer3)
conv_layer4 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv_layer3)

flatten_layer = Flatten()(conv_layer4)

dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=2, activation='softmax')(dense_layer2)

Instructions for updating:
Colocations handled automatically by placer.
(None, 55, 55, 263, 32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
model = Model(inputs=input_layer, outputs=output_layer)

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 61, 61, 275, 1)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 59, 59, 269, 8)    512       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 57, 57, 265, 16)   5776      
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 55, 55, 263, 32)   13856     
_________________________________________________________________
reshape_1 (Reshape)          (None, 55, 55, 8416)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 53, 53, 64)        4847680   
_________________________________________________________________
flatten_1 (Flatten)          (None, 179776)            0         
__________

In [12]:
adam = Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [13]:
filepath = "best-model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [14]:
history = model.fit(x=train_x, y=train_y, batch_size=8, epochs=100, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
375/375 [==============================] - 43s 115ms/step - loss: 4.8751 - acc: 0.6320

Epoch 00001: acc improved from -inf to 0.63200, saving model to best-model.hdf5
Epoch 2/100
375/375 [==============================] - 34s 89ms/step - loss: 9.6485 - acc: 0.3413

Epoch 00002: acc did not improve from 0.63200
Epoch 3/100
375/375 [==============================] - 34s 90ms/step - loss: 10.0345 - acc: 0.3227

Epoch 00003: acc did not improve from 0.63200
Epoch 4/100
375/375 [==============================] - 34s 90ms/step - loss: 10.1201 - acc: 0.3173

Epoch 00004: acc did not improve from 0.63200
Epoch 5/100
375/375 [==============================] - 34s 90ms/step - loss: 10.1636 - acc: 0.3147

Epoch 00005: acc did not improve from 0.63200
Epoch 6/100
375/375 [==============================] - 34s 91ms/step - loss: 10.2911 - acc: 0.3067

Epoch 00006: acc did not improve from 0.63200
Epoch 7/100
375/375 [======================

375/375 [==============================] - 35s 92ms/step - loss: 10.1621 - acc: 0.3147

Epoch 00055: acc did not improve from 0.63200
Epoch 56/100
375/375 [==============================] - 35s 92ms/step - loss: 10.1604 - acc: 0.3147

Epoch 00056: acc did not improve from 0.63200
Epoch 57/100
375/375 [==============================] - 35s 92ms/step - loss: 10.0340 - acc: 0.3227

Epoch 00057: acc did not improve from 0.63200
Epoch 58/100
375/375 [==============================] - 35s 93ms/step - loss: 10.2048 - acc: 0.3120

Epoch 00058: acc did not improve from 0.63200
Epoch 59/100
375/375 [==============================] - 35s 93ms/step - loss: 10.1180 - acc: 0.3173

Epoch 00059: acc did not improve from 0.63200
Epoch 60/100
375/375 [==============================] - 35s 92ms/step - loss: 10.1190 - acc: 0.3173

Epoch 00060: acc did not improve from 0.63200
Epoch 61/100
375/375 [==============================] - 35s 92ms/step - loss: 10.1612 - acc: 0.3147

Epoch 00061: acc did not impro

In [ ]:
# load best weights
model.load_weights("best-model.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
#model.predict(test_x,batch_size=10)
model.evaluate(test_x,test_y,batch_size=8)